In [1]:
%matplotlib widget
# %matplotlib notebook

In [2]:
import numpy as np
import flammkuchen as fl
import napari
from pathlib import Path
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, fixed
import ipywidgets as widgets
from split_dataset import SplitDataset
import json

from bg_atlasapi.bg_atlas import BrainGlobeAtlas
from bg_space import AnatomicalSpace

from quickdisplay import *
import tifffile

## Loading stacks

In [3]:
data_dir = Path(r"\\FUNES\Shared\experiments\E0054_velocity_integrators\v05_imaging_vel_vs_freq")
path_list = list((data_dir).glob('*f[0-9]*'))
path_list

[WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210721_f1'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210721_f2'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210721_f3'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210722_f0'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210722_f2'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210722_f4'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210722_f5'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210722_f6'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity_integrators/v05_imaging_vel_vs_freq/210723_f1'),
 WindowsPath('//FUNES/Shared/experiments/E0054_velocity

### Load reference stack

Specify stack to use as reference, and import stack and metadata to define image resolution.

In [4]:
###FUNCTIONAL IMAGING STACK###

#Specify fish to use as reference
ref_fish = 3
ref_path = path_list[ref_fish]
print('Reference: ', ref_path)

ref = SplitDataset(Path(ref_path / 'anatomy'))[1:, :, :]

#Retrieve resolution
with open(next((ref_path / 'behavior').glob('*metadata*')), "r") as f:
        ref_metadata = json.load(f)

ls_config = ref_metadata['imaging']['microscope_config']['lightsheet']['scanning']
z_tot_span = ls_config["z"]["piezo_max"] - ls_config["z"]["piezo_min"]
n_planes = ls_config["triggering"]["n_planes"]

ref_res = [z_tot_span / n_planes, 0.6, 0.6] #Resolution for xy axis in the LS is 0.6

Reference:  \\FUNES\Shared\experiments\E0054_velocity_integrators\v05_imaging_vel_vs_freq\210722_f0


In [ ]:
###BRAINRENDER REFERENCE###

#Load MPI reference stack
mpi_ref = BrainGlobeAtlas('mpin_zfish_1um')
ref = mpi_ref.additional_references['H2BGCaMP']

#Store resolution
ref_res = mpi_ref.metadata['resolution']

### Load moving stack

Specify fish to register to the reference stack. Directory will be used as output for the resampled images and initial transformation matrix.


In [5]:
#Specify fish to register
fish_path = path_list[6]
print('Morphing: ', fish_path)

#Load functional dataset to align
mov = SplitDataset(Path(fish_path / 'anatomy'))[1:, :, :]

#Retrieve resolution
with open(next((fish_path / 'behavior').glob('*metadata*')), "r") as f:
        fish_metadata = json.load(f)

mov_config = fish_metadata['imaging']['microscope_config']['lightsheet']['scanning']
z_tot_span = mov_config["z"]["piezo_max"] - mov_config["z"]["piezo_min"]
n_planes = mov_config["triggering"]["n_planes"]

mov_res = [z_tot_span / n_planes, 0.6, 0.6]

Morphing:  \\FUNES\Shared\experiments\E0054_velocity_integrators\v05_imaging_vel_vs_freq\210722_f5


## Preprocessing

We first need to make sure both stacks are in the same common space. When regsitering reference brains you may want to downsample their resolutions a bit as mapping between spaces later might be very time-consuming, and not all the resolution information is needed. However, when working with functional stacks, you probably don't want to do any downsampling (especially in z) to preserve as much information as possible.

The most important part is to make sure you define the correct `origin` when creating the `AnatomicaSpace` objects. This will change between setups or depending on the orientation of your stacks. You can check the documentation of `bg-space` [here](https://github.com/brainglobe/bg-space).

In [6]:
# ref_as = AnatomicalSpace('ilp', resolution=ref_res, shape=ref.shape) #LS lab stacks
ref_as = AnatomicalSpace(mpi_ref.orientation, resolution=ref_res, shape=ref.shape) #Brainglobe ref

mov_as = AnatomicalSpace('ilp', resolution=mov_res, shape=mov.shape)

Now we will create a morphing space, and map both of our stacks into it. In this case, despite you could keep the `origin` of your morphing space to be the same as your functional stacks, I suggest sticking to the `'rai'` one if you plan on doing the morphing with ANTsPy. ANTsPy works with [ITK coordinate space](https://www.slicer.org/wiki/Coordinate_systems), so sticking to this option might make your life easier on the long run.

I also suggest upsampling the resolution along the z dimension. This will generate new planes by interpolating the info between adjacent planes. Might not make a lot of sense since it will make everything more computationally-intensive, but in my hands, ANTsPy registration always worked better when passing to it stacks upsampled across the z dimension. (I made this observation when starting to play with morphing parameters, so not 100% convinced about it. Would be happy to know how it works on other people's hands).

In [7]:
#Define morphing space...
morphing_as = AnatomicalSpace('rai', resolution=(1,1,1))

#... and transform references to morphing space
ref_mapped = ref_as.map_stack_to(morphing_as, ref)
mov_mapped = mov_as.map_stack_to(morphing_as, mov)

We will also load the coordinates for our ROIs and map them to the same morphing space. This is essential if you want the final registrations to be applied properly to your ROI coordinates.

In [8]:
#load ROI coordinates...
mov_roi_coords = fl.load(fish_path / 'traces.h5')['coords']

#...and map them as well to the morphing space
mov_roi_coords_mapped = mov_as.map_points_to(morphing_as, mov_roi_coords)

## Safety check
We can check that both stacks and points are in the same space:

In [12]:
interact(plot_side_to_side,
        stack1=fixed(ref_mapped),
        stack2=fixed(mov_mapped),
        depth = widgets.IntSlider(min=0, max=100, step=5, continuous_update=False),
        dim=fixed(2), 
        stack1_title=fixed('Ref fish'),
        stack2_title=fixed('Functional stack'))

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', step=5), Output()), _do…

<function quickdisplay.plot_side_to_side(stack1, stack2, depth, dim, stack1_title='Stack 1', stack2_title='Stack 2')>

In [23]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(np.nanmean(ref_mapped, 2), cmap='gray_r', origin='lower') 
axes[0].set_title('Ref stack')

axes[1].imshow(np.nanmean(mov_mapped, 2), cmap='gray_r', origin='lower') 
axes[1].scatter(mov_roi_coords_mapped[:, 1], mov_roi_coords_mapped[:, 0], c='red', alpha=.0075)
axes[1].set_title('Mov ROIs')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Mov ROIs')

In [ ]:
registration_dir = fish_path / 'registration' / 'antspy'

if not os.path.isdir(registration_dir):
    os.mkdir(registration_dir)
    
#Store mapped stacks
fl.save(registration_dir / "ref_mapped.h5", ref_mapped)
fl.save(registration_dir / "mov_mapped.h5", mov_mapped)

#Store mapped ROI coordinates
fl.save(registration_dir / "mov_roi_coords_mapped.h5", mov_roi_coords_mapped)

### Manual affine transformation

We will start by finding an initial [affine transformation](https://en.wikipedia.org/wiki/Affine_transformation) based on manually-defined points. This is useful in order to get an initial rough estimate of the necessary transformation matrix needed to merge our datasets.

To do so, we will open the reference and the dataset stacks in napari, and [draw some points](https://napari.org/tutorials/fundamentals/points.html) in equivalent regions of the anatomies. Start always with identifiable regions/structures in the anatomy with the lowest resolution, and find these same spots in the reference anatomy. Some good areas to try:
- Extreme brain regions (top-most, rostral telencephalon...)
- Well-defined structures (inferior olive, IPN, Mauthner cells...)
- Centers or borders of larger structures (Habenulae, Optic tectums)
- ...

Make sure you add points in the two different stacks in the same order, or the resulting transformation matrix will make no sense. <br>
And don't go crazy adding infinite points: ~6-8 points scattered around the brain should be more than enough to get a decent initial estimate.

***UPDATE** I realized I prefer to go crazy adding infinite points. Whatever it takes for my initial guess to decently register regions within the brain as well as the general contour of the brain at different depths. Not sure how achievable it is to do both things simultaneously with only this tool, but honnestly I think t will save a bunch of time trying to optimize parameters in ANTsPy.*

In [ ]:
ref_stack, mov_stack = ref_mapped, mov_mapped

#Open stacks in napari
functional_viewer = napari.view_image(mov_stack)
reference_viewer = napari.view_image(ref_stack)

In [ ]:
#Now we retrieve the points we just defined...
functional_points = functional_viewer.layers[1].data
reference_points = reference_viewer.layers[1].data

X = np.pad(reference_points, ((0,0), (0,1)), mode="constant", constant_values=1)
Y = functional_points

#...and we use some math magic to perform an initial fit 
#(this line performs some sort of lineal regression to find a transform matrix between our two arrays of points)
transform_mat =  (np.linalg.pinv(X.T @ X) @ X.T @ Y).T

#Apply transform
transformed = map_affine(mov_stack, transform_mat, ref_stack.shape)

In [ ]:
#Visualize initial approximation
viewer = napari.view_image(ref_stack,  colormap="green")
viewer.add_image(transformed, colormap="magenta", blending="additive")

In [ ]:
#Store the initial transform if happy
fl.save(registration_dir / "initial_transform_mapped.h5", transform_mat)

## ANTsPy Registration¶
Now you should proceed onto the ANTsPy registration. To do so, open the next notebook in the repository (**2_antspy_morphing_linux**) with Jupyter Lab/Jupyter Notebook in your linux terminal. Then load the files you have generated in this notebook to your linux virtual subsystem, and follow the instructions of the next notebook.